In [1]:
from pathlib import Path

from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch_geometric.nn import global_mean_pool
from torchvision import transforms

from layout_gnn.dataset.dataset import RICOTripletsDataset, DATA_PATH
from layout_gnn.dataset import transformations
from layout_gnn.model.model import LayoutGraphModel, CNNNeuralRasterizer
from layout_gnn.model.lightning_module import LayoutGraphModelCNNNeuralRasterizer
from layout_gnn.utils import pyg_triplets_data_collate


In [2]:
dataset = RICOTripletsDataset(triplets=DATA_PATH / "pairs_0_10000.json")
label_mappings = {k: i for i, k in enumerate(dataset.label_color_map)}
dataset.transform = transform=transforms.Compose([
    transformations.process_data,
    transformations.normalize_bboxes,
    transformations.add_networkx,
    transformations.RescaleImage(256, 256, allow_missing_image=True),
    transformations.ConvertLabelsToIndexes(
        node_label_mappings=label_mappings,
        # edge_label_mappings={"parent_of": 0, "child_of": 1},
    ),
    transformations.convert_graph_to_pyg,
])

data_loader = DataLoader(dataset=dataset, batch_size=8, collate_fn=pyg_triplets_data_collate)
model = LayoutGraphModelCNNNeuralRasterizer(
    num_labels=len(label_mappings) + 1, 
    cnn_output_dim=3,
    cnn_output_size=256,
)
trainer = Trainer(default_root_dir=DATA_PATH)

/home/drz/Layout-GNN/.venv/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
trainer.fit(model, data_loader)

/home/drz/Layout-GNN/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:107: UserWarning: attribute 'readout' removed from hparams because it cannot be pickled
  rank_zero_warn(f"attribute '{k}' removed from hparams because it cannot be pickled")
/home/drz/Layout-GNN/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name                | Type                          | Params
----------------------------------------------------------------------
0 | encoder             | LayoutGraphModel              | 42.3 K
1 | decoder             | CNNNeuralRasterizer           | 1.2 M 
2 | triplet_loss        | TripletMarginWithDistanceLoss | 0     
3 | reconstruction_loss | MSELoss                       | 0     
--------------------------

Training: 0it [00:00, ?it/s]

/home/drz/Layout-GNN/.venv/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


RuntimeError: Found dtype Double but expected Float

In [ ]:
type(model.hparams)

pytorch_lightning.utilities.parsing.AttributeDict